# Brute Forcing Kryptos — An Attempt

Attempting to brute force the unsolved Kryptos cipher (K4 segment).

Using Python because I'm rusty as fuck with C at the moment. Plus I've recently used a lot of Python with GPU acceleration in my [CS thesis](https://github.com/vinivosh/ufu-tcc2), so it'll be very useful here and won't require me to learn new stuff — always a great bonus for a lazy lad.

I'll try using [Numba](https://numba.pydata.org/)'s [jit compiler](https://numba.readthedocs.io/en/stable/user/jit.html) to accelerate the process this as much as possible. Preferably even Numba's [CUDA support](https://numba.readthedocs.io/en/stable/cuda/overview.html) if it works.

# The Segments

Of note: **K2** here is the corrected version. For the version in the actual sculpture, with the omission error, just substitute the `ESWJL` near the end by `EWJL`

In [1]:
K1 = '''EMUFPHZLRFAXYUSDJKZLDKRNSHGNFIVJ
YQTQUXQBQVYUVLLTREVJYQTMKYRDMFD'''

K2 = '''VFPJUDEEHZWETZYVGWHKKQETGFQJNCE
GGWHKK?DQMCPFQZDQMMIAGPFXHQRLG
TIMVMZJANQLVKQEDAGDVFRPJUNGEUNA
QZGZLECGYUXUEENJTBJLBQCRTBJDFHRR
YIZETKZEMVDUFKSJHKFWHKUWQLSZFTI
HHDDDUVH?DWKBFUFPWNTDFIYCUQZERE
EVLDKFEZMOQQJLTTUGSYQPFEUNLAVIDX
FLGGTEZ?FKZBSFDQVGOGIPUFXHHDRKF
FHQNTGPUAECNUVPDJMQCLQUMUNEDFQ
ELZZVRRGKFFVOEEXBDMVPNFQXEZLGRE
DNQFMPNZGLFLPMRJQYALMGNUVPDXVKP
DQUMEBEDMHDAFMJGZNUPLGESWJLLAETG'''

K3 = '''ENDYAHROHNLSRHEOCPTEOIBIDYSHNAIA
CHTNREYULDSLLSLLNOHSNOSMRWXMNE
TPRNGATIHNRARPESLNNELEBLPIIACAE
WMTWNDITEENRAHCTENEUDRETNHAEOE
TFOLSEDTIWENHAEIOYTEYQHEENCTAYCR
EIFTBRSPAMHHEWENATAMATEGYEERLB
TEEFOASFIOTUETUAEOTOARMAEERTNRTI
BSEDDNIAAHTTMSTEWPIEROAGRIEWFEB
AECTDDHILCEIHSITEGOEAOSDDRYDLORIT
RKLMLEHAGTDHARDPNEOHMGFMFEUHE
ECDMRIPFEIMEHNLSSTTRTVDOHW?'''

K4 = '''OBKR
UOXOGHULBSOLIFBBWFLRVQQPRNGKSSO
TWTQSJQSSEKZZWATJKLUDIAWINFBNYP
VTTMZFPKWGDKZXTJCDIGKUHUAUEKCAR'''

# print(f'K1:\n{K1}\n\nK2:\n{K2}\n\nK3:\n{K3}\n\nK4:\n{K4}\n\n')

# Dictionary and Evaluation

How to know if the decrypted message candidates make sense and are not just random gibberish? Using some known patterns in the english language!

> I got this idea from [this great experiment](https://github.com/desgeeko/kryptos/blob/main/Kryptos.ipynb), so all credit goes to GitHub user **[desgeeko](https://github.com/desgeeko)**.

## Reading Files (Google Web Trillion Word Corpus)

In [2]:
import os
import time
from urllib.request import urlopen

import numpy as np
import pandas as pd
# from numba import jit

def download_if_needed(file_path, url):
    '''Downloads file from `url`, if it doesn't already exists in `filePath`'''

    if os.path.exists(file_path): return

    with urlopen(url) as f:
        html = f.read().decode('utf-8')
    with open(file_path, 'w') as f:
        f.write(html)

file_count_1w = 'count_1w.txt'
file_count_2l = 'count_2l.txt'
url_base = 'https://norvig.com/ngrams/'

download_if_needed(file_count_1w, url_base + file_count_1w)
download_if_needed(file_count_2l, url_base + file_count_2l)

# Reading ngrams files
with open(file_count_1w, 'r') as f:
    count_1w = pd.read_csv(f, names=['word', 'count'], sep='\t')
with open(file_count_2l, 'r') as f:
    count_2l = pd.read_csv(f, names=['bigram', 'count'], sep='\t')

# Normalizing bigram count, to help calculations
minBigramCount = count_2l['count'].min()
count_2l['count'] = (count_2l['count'] / minBigramCount).astype(np.uint32)

print(count_1w)
print(count_2l)

# Converting count_2l to a dictionary
count_2l = {bigram.upper(): count for (bigram, count) in count_2l.to_numpy()}
# print(count_2l)

           word        count
0           the  23135851162
1            of  13151942776
2           and  12997637966
3            to  12136980858
4             a   9081174698
...         ...          ...
333328    gooek        12711
333329   gooddg        12711
333330  gooblle        12711
333331   gollgo        12711
333332    golgw        12711

[333333 rows x 2 columns]
    bigram  count
0       in  47154
1       th  46594
2       er  41698
3       re  38010
4       he  37250
..     ...    ...
671     qy      2
672     zq      2
673     jx      1
674     qz      1
675     jq      1

[676 rows x 2 columns]


## Evaluation Function

In [20]:
def is_it_english(text:str, use_geo_mean=False, ignore_invalid_chars=True):
    '''Calculates and returns an integer score that represents how high is the chance of the string passed in `text` being an english text'''

    bigrams = [text[i:i+2].upper() for i in range(len(text) - 1)]
    # print(bigrams)
    total = 0
    bigramsChecked = 0

    for bigram in bigrams:
        score = count_2l.get(bigram, None)
        # print(score)
        if score is None: continue
        bigramsChecked += 1

        if use_geo_mean: total += np.log(score)
        else: total += score

    if use_geo_mean: return round(np.exp(total / (bigramsChecked if ignore_invalid_chars else len(text))))
    return round(total / (bigramsChecked if ignore_invalid_chars else len(text)))

testTexts = [
    'A CAT NAP A DAY LATE AND A DOLLAR SHORT LOVE BIRDS A LOT ON ONES PLATE A BITE AT THE CHERRY',
    'TEST PHRASE THAT IS COMPLETELY WRITTEN IN ENGLISH LETS SEE IF THIS GOES AS PLANNED',

    'FRASE DE PRUEBA QUE ESTA COMPLETAMENTE ESCRITA EN ESPANOL A VER SI VA SEGUN LO PLANIFICADO',
    'PHRASE DE TEST ENTIEREMENT ECRITE EN FRANCAIS VOYONS SI CELA SE PASSE COMME PREVU',
    'TESTZIN DIE VOLLEDIG IN HET NEDERLANDS IS GESCHREVEN LATEN WE KIJKEN OF DIT GAAT ZOALS PLANNEN',
    'FRASE EM PORTUGUES PARA TESTAR COMO O SCORE FICA SE TESTARMOS OUTRA LINGUA BANANAS',

    'WPRYCSWDUWQXAYKMOKUZLVOSIPPTGTIIDZXQYCOCSLIAGPEPFJEXFVVMMOPLRJYATIISHMPNFJFIEWLVQ',
    'CJODRMQNNKCEGPOXUUQOXISVUIRIDLXMLXFKWREBKWEZMRCRPXNNFKOIOEKOXBCOUZGGIORDTNXYNRMTR',
    'HQLVEOWOTLEZNDGYFFPLTCJCIOGGFWAMYZOXHIGJMFSLEPHLVGRDIGQURPAIIFLWICWYSHZWCIMRJWVUB',
    'XUCUSMSSROGJKIGUPGSBGSIKINELYBHOALXCQYTGMRKDWJUXDUOANVEWHIWDIMGKMKKUSIYFQAGBLWTDF',

    '''The Lord is my shepherd; I shall not want. / He maketh me to lie down in green pastures: he leadeth me beside the still waters. / He restoreth my soul: he leadeth me in the paths of righteousness for his name's sake. / Yea, though I walk through the valley of the shadow of death, I will fear no evil: for thou art with me; thy rod and thy staff they comfort me. / Thou preparest a table before me in the presence of mine enemies: thou anointest my head with oil; my cup runneth over. / Surely goodness and mercy shall follow me all the days of my life: and I will dwell in the house of the Lord for ever.''',

    '''I MET A TRAVELLER from an antique land / Who said: Two vast and trunkless legs of stone / Stand in the desert. Near them on the sand, / Half sunk, a shattered visage lies, whose frown / And wrinkled lip and sneer of cold command / Tell that its sculptor well those passions read / Which yet survive, stamped on these lifeless things, / The hand that mocked them and the heart that fed. / And on the pedestal these words appear: / "My name is Ozymandias, King of Kings: / Look on my works, ye mighty, and despair!" / Nothing beside remains. Round the decay / Of that colossal wreck, boundless and bare, / The lone and level sands stretch far away.''',

    '''And why take ye thought for raiment? Consider the lilies of the field, how they grow; they toil not, neither do they spin: / And yet I say unto you, That even Solomon in all his glory was not arrayed like one of these. / Wherefore, if God so clothe the grass of the field, which today is, and tomorrow is cast into the oven, shall he not much more clothe you, O ye of little faith? Therefore, take no thought, saying, What shall we eat? or, What shall we drink? or, Wherewithal shall we be clothed? / For your heavenly Father knoweth that ye have need of all these things . . . / Take therefore no thought for the morrow: for the morrow shall take thought for the things of itself. Sufficient unto the day is the evil thereof.''',

    '''I wish I loved the Human Race; / I wish I loved its silly face; / I wish I liked the way it walks; / I wish I liked the way it talks; / And when I'm introduced to one, / I wish I thought "What Jolly Fun!"''',

    '''Monday's child is fair of face, / Tuesday's child is full of grace, / Wednesday's child is full of woe, / Thursday's child has far to go, / Friday's child is loving and giving, / Saturday's child works hard for its living, / And a child that's born on the Sabbath day / Is fair and wise and good and gay.''',

    '''A grasshopper spent the summer hopping about in the sun and singing to his heart's content. One day, an ant went hurrying by, looking very hot and weary. / "Why are you working on such a lovely day?" said the grasshopper. / "I'm collecting food for the winter," said the ant, "and I suggest you do the same." And off she went, helping the other ants to carry food to their store. The grasshopper carried on hopping and singing. When winter came the ground was covered with snow. The grasshopper had no food and was hungry. So he went to the ants and asked for food. / "What did you do all summer when we were working to collect our food?" said one of the ants. / "I was busy hopping and singing," said the grasshopper. / "Well," said the ant, "if you hop and sing all summer, and do no work, then you must starve in the winter."''',

    '''One day the Hare laughed at the short feet and slow speed of the Tortoise. The Tortoise replied: / "You may be as fast as the wind, but I will beat you in a race!" / The Hare thought this idea was impossible and he agreed to the proposal. It was agreed that the Fox should choose the course and decide the end. / The day for the race came, and the Tortoise and Hare started together. / The Tortoise never stopped for a moment, walking slowly but steadily, right to the end of the course. The Hare ran fast and stopped to lie down for a rest. But he fell fast asleep. Eventually, he woke up and ran as fast as he could. But when he reached the end, he saw the Tortoise there already, sleeping comfortably after her effort.''',

    '''Kaze ni fukarete anata no kami/ Shinobikomu no / Jitensha kara kiritoru / Atarashii keshiki ni kusari o kakeru you ni / Koshi ni soeta atatakasa mo / Anata ni nara ubawarete mo ii / Shizuka na machi / Teishoku-ya no natsukashisa yo / Anata no tonari de ikite iru / Yume no you ni / Izure yuragu ayausa mo / Haru no hikari ni amaku tokete yuku''',

    '''Bengara iro no ganseki / Yume utsutsu na azuki iro no sora / Dareka ga aruita ato / Arashi no ato / Chikyuu ga aojiroku hikatte mieru / Aikawarazu no taiyou wa / Koko demo mada hikari tsuzuketeru / Taiki o oou funjin wa / Itsu no hi ni ka / Moeta okane no you natsukashii / Myoujou yo dou omou kai / Furoshiki no okome wa soko o tsuite / Ryuusei o kazoeru no mo akite / Kusarenai karada wa kaeshitai / Kaesu hoshi wa towa no kana tada''',

    '''yoba reta hito wa tayasuku noborete shimau tsuki no oka ano ko wa mada watashi-tachi ikutsu mo yakusoku o shita mama arashi no yoru ni fukukaze hoho wa nurete namidaame ano ko no eyes eien ni sakarainagara ikite kita no sora o saite kagayaku kage hoshi ni chikai kiete yuku hohoemi wa marui tsuki ni natte oka ni hisonda no''',
]

runs = 1 # 333340 // len(testTexts)
timings = []

for text in testTexts:
    for runIdx in range(runs):
        text = text.upper()
        startTime = time.perf_counter_ns()
        score = is_it_english(text)
        timings.append(time.perf_counter_ns() - startTime)
    print(f'"{text}"\n{score}\n')

timings = np.array(timings, np.int64)
print(f'Total time to score {runs * len(testTexts)} cipher texts = {timings.sum() * 1e-6} ms')
print(f'Avg time to score a cipher text = {timings.mean()} \u00B1 {timings.std()} ns')

# Testing some bigger and classic english texts
testTexts = [
    #
]

"A CAT NAP A DAY LATE AND A DOLLAR SHORT LOVE BIRDS A LOT ON ONES PLATE A BITE AT THE CHERRY"
17783

"TEST PHRASE THAT IS COMPLETELY WRITTEN IN ENGLISH LETS SEE IF THIS GOES AS PLANNED"
17392

"FRASE DE PRUEBA QUE ESTA COMPLETAMENTE ESCRITA EN ESPANOL A VER SI VA SEGUN LO PLANIFICADO"
14284

"PHRASE DE TEST ENTIEREMENT ECRITE EN FRANCAIS VOYONS SI CELA SE PASSE COMME PREVU"
17025

"TESTZIN DIE VOLLEDIG IN HET NEDERLANDS IS GESCHREVEN LATEN WE KIJKEN OF DIT GAAT ZOALS PLANNEN"
16374

"FRASE EM PORTUGUES PARA TESTAR COMO O SCORE FICA SE TESTARMOS OUTRA LINGUA BANANAS"
16928

"WPRYCSWDUWQXAYKMOKUZLVOSIPPTGTIIDZXQYCOCSLIAGPEPFJEXFVVMMOPLRJYATIISHMPNFJFIEWLVQ"
4185

"CJODRMQNNKCEGPOXUUQOXISVUIRIDLXMLXFKWREBKWEZMRCRPXNNFKOIOEKOXBCOUZGGIORDTNXYNRMTR"
3825

"HQLVEOWOTLEZNDGYFFPLTCJCIOGGFWAMYZOXHIGJMFSLEPHLVGRDIGQURPAIIFLWICWYSHZWCIMRJWVUB"
3927

"XUCUSMSSROGJKIGUPGSBGSIKINELYBHOALXCQYTGMRKDWJUXDUOANVEWHIWDIMGKMKKUSIYFQAGBLWTDF"
4606

"THE LORD IS MY SHEPHERD; I SHALL NOT WANT. / HE MAKETH ME T

### Benchmarking

The evaluation function was benchmarked and yielded the following results

Using arithmetic mean (regular average operation):
```
Total time to score 333340 cipher texts = 2962.16394 ms
Avg time to score a cipher text = 8886.314093718125 ± 1719.0663828394152 ns

```

Using geometric mean:
```
Total time to score 333340 cipher texts = 18414.80903 ms
Avg time to score a cipher text = 55243.32222355553 ± 5830.699753181231 ns

```

# Brute Forcing K1